# Генерация предложений для машинки 

на ржя и на русском

В этой тетрадке я генерирую предложения на русском и на ржя. Предложения по типу SVO для переходных глаголов и предложения по типу SV для непереходных глаголов.

In [1]:
from pprint import pprint
import pandas as pd
import json
from pymystem3 import Mystem
import re

mystem = Mystem()

## Кусок ниже нельзя трогать

он работает тупо два часа и пересоздает файл

In [31]:
#words = pd.read_csv("animation.csv", encoding="utf-8", delimiter=";", names=["id", "text"])
#words.tail()

,id,text
3586,4067,яйцо
3587,4068,япония
3588,4069,ярмарка
3589,4070,ясли
3590,4071,ящик


In [34]:
#words['analysis'] = words['text'].apply(mystem.analyze)
#words['analysis']

0       [{'text': 'А', 'analysis': [{'wt': 0.011249589...
1       [{'text': 'Б', 'analysis': [{'wt': 0.998029947...
2       [{'text': 'В', 'analysis': [{'wt': 8.212236024...
3       [{'text': 'Г', 'analysis': [{'wt': 1, 'lex': '...
4       [{'text': 'Д', 'analysis': [{'wt': 1, 'lex': '...
                              ...                        
3586    [{'text': 'яйцо', 'analysis': [{'wt': 1, 'lex'...
3587    [{'text': 'япония', 'analysis': [{'wt': 1, 'le...
3588    [{'text': 'ярмарка', 'analysis': [{'wt': 1, 'l...
3589    [{'text': 'ясли', 'analysis': [{'wt': 1, 'lex'...
3590    [{'text': 'ящик', 'analysis': [{'wt': 1, 'lex'...
Name: analysis, Length: 3591, dtype: object

In [35]:
#words.to_csv("animation_m.csv", encoding="utf-8")

## До этого момента ничего не трогаем

In [2]:
words = pd.read_csv("animation_m.csv", encoding="utf-8")
words.tail()

,Unnamed: 0,id,text,analysis
3586,3586,4067,яйцо,"[{'text': 'яйцо', 'analysis': [{'wt': 1, 'lex'..."
3587,3587,4068,япония,"[{'text': 'япония', 'analysis': [{'wt': 1, 'le..."
3588,3588,4069,ярмарка,"[{'text': 'ярмарка', 'analysis': [{'wt': 1, 'l..."
3589,3589,4070,ясли,"[{'text': 'ясли', 'analysis': [{'wt': 1, 'lex'..."
3590,3590,4071,ящик,"[{'text': 'ящик', 'analysis': [{'wt': 1, 'lex'..."


In [3]:
# Проверила, что все слова кончаются на \n, его можно стрипнуть из этой колонки
for el in words.analysis:
    if len(el) < 2:
        print(len(el))

In [4]:
for el in words.analysis:
    try:
        an = json.loads(el.replace("'", '"'))
    except:
        print(el)

In [5]:
words['analysis'] = words['analysis'].apply(lambda x: json.loads(x.replace("'", '"'))[:-1])

In [6]:
# test
for i in range(300, 350):
    for el in words['analysis'][i]:
        if el['text'].isalnum():
            print(el)
    print("===================")

{'text': 'в', 'analysis': [{'wt': 0.9999917746, 'lex': 'в', 'gr': 'PR='}]}
{'text': 'то', 'analysis': [{'wt': 0.017226537689999998, 'lex': 'тот', 'gr': 'APRO=(вин,ед,сред|им,ед,сред)'}]}
{'text': 'время', 'analysis': [{'wt': 1, 'lex': 'время', 'gr': 'S,сред,неод=(вин,ед|им,ед)'}]}
{'text': 'как', 'analysis': [{'wt': 0.3756069839, 'lex': 'как', 'gr': 'ADVPRO='}]}
{'text': 'вечность', 'analysis': [{'wt': 1, 'lex': 'вечность', 'gr': 'S,жен,неод=(вин,ед|им,ед)'}]}
{'text': 'навсегда', 'analysis': [{'wt': 1, 'lex': 'навсегда', 'gr': 'ADV='}]}
{'text': 'вечный', 'analysis': [{'wt': 1, 'lex': 'вечный', 'gr': 'A=(вин,ед,полн,муж,неод|им,ед,полн,муж)'}]}
{'text': 'воскресенье', 'analysis': [{'wt': 1, 'lex': 'воскресение', 'gr': 'S,сред,неод=(пр,ед|вин,ед|им,ед)'}]}
{'text': 'временный', 'analysis': [{'wt': 1, 'lex': 'временный', 'gr': 'A=(вин,ед,полн,муж,неод|им,ед,полн,муж)'}]}
{'text': 'пока', 'analysis': [{'wt': 0.3090040088, 'lex': 'пока', 'gr': 'ADV='}]}
{'text': 'выходной', 'analysis': [{

In [7]:
pos_column = []
for i in range(len(words)):
    pos_all = []
    for el in words['analysis'][i]:
        #print(el)
        if el['text'] in 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ':
            pos_all.append('DACTYL')
        elif el['text'].isalpha():
            if el['analysis']:
                almost_pos = el['analysis'][0]['gr'].split(',')[0]
                if '=' in almost_pos:
                    almost_pos = almost_pos.split('=')[0]
                pos_all.append(almost_pos)
            else:
                pos_all.append(None)
        elif re.match(r'\d+', el['text']):
            pos_all.append('NUM')
        else:
            pos_all.append(None)
    if 'V' in pos_all:
        pos_column.append('V')
    elif 'PR' in pos_all and len(pos_all) > 1:
        pos_column.append(None)
    elif 'PR' in pos_all and len(pos_all) == 1:
        pos_column.append('PR')
    elif 'S' in pos_all and len(pos_all) > 1 and 'NUM' in pos_all:
        pos_column.append('NUM')
    elif pos_all.count('S') == 2 or ('S' in pos_all and 'A' in pos_all):
        pos_column.append('S')
    elif 'S' in pos_all and 'PART' in pos_all:
        pos_column.append('V')
    elif 'S' in pos_all and len(pos_all) > 1:
        pos_column.append(None)
    elif 'S' in pos_all:
        pos_column.append('S')
    elif 'SPRO' in pos_all:
        pos_column.append('SPRO')
    elif 'ADV' in pos_all:
        pos_column.append('ADV')
    elif 'NUM' in pos_all:
        pos_column.append('NUM')
    elif 'A' in pos_all:
        pos_column.append('A')
    elif 'DACTYL' in pos_all:
        pos_column.append('DACTYL')
    elif 'ANUM' in pos_all:
        pos_column.append('ANUM')
    elif 'APRO' in pos_all:
        pos_column.append('APRO')
    elif 'ADVPRO' in pos_all and 'PART' in pos_all:
        pos_column.append('ADV')
    elif 'ADVPRO' in pos_all:
        pos_column.append('ADVPRO')
    elif 'PART' in pos_all:
        pos_column.append('PART')
    elif 'CONJ' in pos_all:
        pos_column.append('CONJ')
    elif 'INTJ' in pos_all:
        pos_column.append('INTJ')
    elif words['analysis'][i][0]['text'] == 'жадный2':
        pos_column.append('A')
    elif words['analysis'][i][0]['text'] == 'чуть-чуть':
        pos_column.append('ADV')
    else:
        pos_column.append('S')

In [8]:
len(pos_column) == len(words)

True

Проверили, что по длине колонка правильно собралась, можно добавлять к данным:

In [9]:
words['pos'] = pos_column
words.tail()

,Unnamed: 0,id,text,analysis,pos
3586,3586,4067,яйцо,"[{'text': 'яйцо', 'analysis': [{'wt': 1, 'lex'...",S
3587,3587,4068,япония,"[{'text': 'япония', 'analysis': [{'wt': 1, 'le...",S
3588,3588,4069,ярмарка,"[{'text': 'ярмарка', 'analysis': [{'wt': 1, 'l...",S
3589,3589,4070,ясли,"[{'text': 'ясли', 'analysis': [{'wt': 1, 'lex'...",S
3590,3590,4071,ящик,"[{'text': 'ящик', 'analysis': [{'wt': 1, 'lex'...",S


In [10]:
# сохранили чекпоинт
words.to_csv("animation_pos.csv", encoding="utf-8")

### Добавляем колонку про транзитивность глаголов

In [10]:
tr_column = []
for i in range(len(words)):
    if words['pos'][i] == 'V':
        if 'пе=' in words['analysis'][i][0]['analysis'][0]['gr']:
            tr_column.append('transitive')
        else:
            tr_column.append('intransitive')
    else:
        tr_column.append(None)

In [11]:
len(tr_column) == len(words)

True

In [12]:
words['transitivity'] = tr_column

In [13]:
words.tail(50)

,Unnamed: 0,id,text,analysis,pos,transitivity
3541,3541,4016,щедрый,"[{'text': 'щедрый', 'analysis': [{'wt': 1, 'le...",A,None
3542,3542,4017,щека,"[{'text': 'щека', 'analysis': [{'wt': 1, 'lex'...",S,None
3543,3543,4018,щекотать,"[{'text': 'щекотать', 'analysis': [{'wt': 1, '...",V,transitive
3544,3544,4019,щелкать пальцами,"[{'text': 'щелкать', 'analysis': [{'wt': 1, 'l...",V,transitive
3545,3545,4020,щепотка,"[{'text': 'щепотка', 'analysis': [{'wt': 1, 'l...",S,None
3546,3546,4021,щипать,"[{'text': 'щипать', 'analysis': [{'wt': 1, 'le...",V,transitive
3547,3547,4022,"щипцы, клещи","[{'text': 'щипцы', 'analysis': [{'wt': 0.78982...",S,None
3548,3548,4023,эвакуировать,"[{'text': 'эвакуировать', 'analysis': [{'wt': ...",V,transitive
3549,3549,4024,эгоист,"[{'text': 'эгоист', 'analysis': [{'wt': 1, 'le...",S,None
3550,3550,4025,эгоистичный,"[{'text': 'эгоистичный', 'analysis': [{'wt': 1...",A,None


In [14]:
# сохранили чекпоинт
words.to_csv("animation_pos2.csv", encoding="utf-8")

### Добавляем колонку про одушевленность существительных

In [14]:
act_column = []
for i in range(len(words)):
    if words['pos'][i] == 'S' or words['pos'][i] == 'SPRO':
        if words['analysis'][i][0]['analysis']:
            print(words['analysis'][i][0])
            ## Надо брать не нулевое слово, а то у которого в gr тег S !!
            if 'неод' in words['analysis'][i][0]['analysis'][0]['gr']:
                act_column.append('inan')
            else:
                act_column.append('anim')
        else:
             act_column.append('inan')
    else:
        act_column.append(None)

{'text': 'цвет', 'analysis': [{'wt': 0.9999979138, 'lex': 'цвет', 'gr': 'S,муж,неод=(вин,ед|им,ед)'}]}
{'text': 'Мама', 'analysis': [{'wt': 1, 'lex': 'мама', 'gr': 'S,жен,од=им,ед'}]}
{'text': 'Папа', 'analysis': [{'wt': 0.9996149540000001, 'lex': 'папа', 'gr': 'S,муж,од=им,ед'}]}
{'text': 'Дядя', 'analysis': [{'wt': 1, 'lex': 'дядя', 'gr': 'S,муж,од=им,ед'}]}
{'text': 'Дочь', 'analysis': [{'wt': 1, 'lex': 'дочь', 'gr': 'S,жен,од=(вин,ед|им,ед)'}]}
{'text': 'Сын', 'analysis': [{'wt': 1, 'lex': 'сын', 'gr': 'S,муж,од=им,ед'}]}
{'text': 'Брат', 'analysis': [{'wt': 1, 'lex': 'брат', 'gr': 'S,муж,од=им,ед'}]}
{'text': 'Сестра', 'analysis': [{'wt': 1, 'lex': 'сестра', 'gr': 'S,жен,од=им,ед'}]}
{'text': 'Ребенок', 'analysis': [{'wt': 1, 'lex': 'ребенок', 'gr': 'S,муж,од=им,ед'}]}
{'text': 'Тетя', 'analysis': [{'wt': 1, 'lex': 'тетя', 'gr': 'S,жен,од=им,ед'}]}
{'text': 'Племянник', 'analysis': [{'wt': 1, 'lex': 'племянник', 'gr': 'S,муж,од=им,ед'}]}
{'text': 'Племянница', 'analysis': [{'wt': 

{'text': 'панцирь', 'analysis': [{'wt': 1, 'lex': 'панцирь', 'gr': 'S,муж,неод=(вин,ед|им,ед)'}]}
{'text': 'папа', 'analysis': [{'wt': 0.9996149540000001, 'lex': 'папа', 'gr': 'S,муж,од=им,ед'}]}
{'text': 'папка', 'analysis': [{'wt': 1, 'lex': 'папка', 'gr': 'S,жен,неод=им,ед'}]}
{'text': 'папоротник', 'analysis': [{'wt': 1, 'lex': 'папоротник', 'gr': 'S,муж,неод=(вин,ед|им,ед)'}]}
{'text': 'пара', 'analysis': [{'wt': 0.8380485773, 'lex': 'пара', 'gr': 'S,жен,неод=им,ед'}]}
{'text': 'парад', 'analysis': [{'wt': 1, 'lex': 'парад', 'gr': 'S,муж,неод=(вин,ед|им,ед)'}]}
{'text': 'парадокс', 'analysis': [{'wt': 1, 'lex': 'парадокс', 'gr': 'S,муж,неод=(вин,ед|им,ед)'}]}
{'text': 'паразит', 'analysis': [{'wt': 0.8268563151, 'lex': 'паразит', 'gr': 'S,муж,од=им,ед'}]}
{'text': 'парашют', 'analysis': [{'wt': 1, 'lex': 'парашют', 'gr': 'S,муж,неод=(вин,ед|им,ед)'}]}
{'text': 'Париж', 'analysis': [{'wt': 1, 'lex': 'париж', 'gr': 'S,гео,муж,неод=(вин,ед|им,ед)'}]}
{'text': 'парк', 'analysis': [{'w

In [15]:
len(act_column) == len(words)

True

In [16]:
words['animacy'] = act_column

In [17]:
words.tail(50)

,Unnamed: 0,id,text,analysis,pos,transitivity,animacy
3541,3541,4016,щедрый,"[{'text': 'щедрый', 'analysis': [{'wt': 1, 'le...",A,None,None
3542,3542,4017,щека,"[{'text': 'щека', 'analysis': [{'wt': 1, 'lex'...",S,None,inan
3543,3543,4018,щекотать,"[{'text': 'щекотать', 'analysis': [{'wt': 1, '...",V,transitive,None
3544,3544,4019,щелкать пальцами,"[{'text': 'щелкать', 'analysis': [{'wt': 1, 'l...",V,transitive,None
3545,3545,4020,щепотка,"[{'text': 'щепотка', 'analysis': [{'wt': 1, 'l...",S,None,inan
3546,3546,4021,щипать,"[{'text': 'щипать', 'analysis': [{'wt': 1, 'le...",V,transitive,None
3547,3547,4022,"щипцы, клещи","[{'text': 'щипцы', 'analysis': [{'wt': 0.78982...",S,None,inan
3548,3548,4023,эвакуировать,"[{'text': 'эвакуировать', 'analysis': [{'wt': ...",V,transitive,None
3549,3549,4024,эгоист,"[{'text': 'эгоист', 'analysis': [{'wt': 1, 'le...",S,None,anim
3550,3550,4025,эгоистичный,"[{'text': 'эгоистичный', 'analysis': [{'wt': 1...",A,None,None


### Делаю csv наречий для Вики

In [211]:
words[words['pos'] == ('ADV' or 'ADVPRO')][['id', 'text', 'pos']].to_csv("adverbs.csv", encoding="utf-8")

## Генерируем предложения

Сначала на ржя

Готовим списки жестов с подходящими нам морфологическими тегами

In [18]:
from tqdm import tqdm

In [19]:
subjects = [el.strip(' ') for el in words[words['animacy'] == 'anim']['text']]

In [20]:
subjects[20:50]

['девочка',
 'жестовое имя',
 'гений',
 'лицемер',
 'резкая боль',
 'администратор',
 'воспитатель',
 'высшее образование',
 'маркер',
 'президент',
 'сверстник',
 'студент',
 'алкоголик',
 'ветеран',
 'все',
 'акробат',
 'акула',
 'алкаш',
 'аллигатор',
 'ангел',
 'арбитр',
 'бабочка',
 'баран',
 'бегемот',
 'бегунок',
 'биполярное расстройство',
 'больное горло',
 'больной',
 'босс шишка',
 'буржуа']

In [21]:
objects = [el.strip(' ') for el in words[words['animacy'] == 'inan']['text']]

In [22]:
objects[:5]

['цвет', 'будущие родители', 'миллиард', 'ноль', 'боль']

In [23]:
tr_verbs = [el.strip(' ') for el in words[words['transitivity'] == 'transitive']['text']]

In [24]:
tr_verbs[:5]

['говорить на жестовом языке',
 'беспокоящий',
 'чувствовать',
 'подожди',
 'задерживать']

In [25]:
intr_verbs = [el.strip(' ') for el in words[words['transitivity'] == 'intransitive']['text']]

In [26]:
intr_verbs[:5]

['бояться', 'торопиться', 'занято', 'совпадать', 'уйти из жизни']

#### SV с непереходными глаголами

In [27]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [31]:
with open('rsl_sv.txt', 'w', encoding='utf-8') as f:
    for s in tqdm(subjects):
        person = None
        m = re.search(r'.*(\d.*?)( |,)', str(morph.parse(s)[0].tag))
        if m:
            person = m.group(1)
            print(person)
        for iv in intr_verbs:
            inflected = None
            verb = morph.parse(iv)[0]
            if person: 
                inflected = verb.inflect({"VERB", person})
            else:
                inflected = verb.inflect({"VERB", "3per"})
            #print("INFLECTED ", inflected)
            if inflected:
                f.write(s + ' ' + iv + '\t' + s.capitalize() + ' ' + inflected.word + '.\n')
            else:
                f.write(s + ' ' + iv + '\t' + s.capitalize() + ' ' + iv + '.\n')

 59%|███████████████████████████████████████████████▌                                | 148/249 [00:25<00:17,  5.84it/s]

1per


 61%|████████████████████████████████████████████████▌                               | 151/249 [00:26<00:17,  5.62it/s]

1per


 91%|████████████████████████████████████████████████████████████████████████▉       | 227/249 [00:38<00:03,  6.37it/s]

2per


100%|███████████████████████████████████████████████████████████████████████████████▋| 248/249 [00:42<00:00,  6.17it/s]

1per


100%|████████████████████████████████████████████████████████████████████████████████| 249/249 [00:42<00:00,  5.34it/s]


In [28]:
# test
verb = morph.parse('уезжать')[0]
print(verb)
inflected = verb.inflect({"VERB", "3per"})
print(inflected.word)

Parse(word='уезжать', tag=OpencorporaTag('INFN,impf,intr'), normal_form='уезжать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'уезжать', 15, 0),))
уезжает


In [263]:
# test
str(morph.parse("президент")[0].tag)

'NOUN,anim,masc sing,nomn'

In [33]:
# test
re.search(r'.*(\d.*?)( |,)', str(morph.parse("они")[0].tag)).group(1)

'3per'

#### SVO с переходными глаголами и объектами

In [29]:
len(subjects)

249

In [35]:
with open('rsl_svo_50.tsv', 'w', encoding='utf-8') as f:
    for s in tqdm(subjects[247:249]):
        person = None
        m = re.search(r'.*(\d.*?)( |,)', str(morph.parse(s)[0].tag))
        if m:
            person = m.group(1)
        for iv in tr_verbs:
            inflected = None
            verb = morph.parse(iv)[0]
            if person: 
                inflected = verb.inflect({"VERB", person})
            else:
                inflected = verb.inflect({"VERB", "3per"})
            for o in objects:
                obj = morph.parse(o)[0]
                obj_acc = obj.inflect({'accs'})
                if inflected and obj_acc:
                    f.write(s + ' ' + iv + ' ' + o + '\t' + s.capitalize() + ' ' + inflected.word +\
                            ' ' + obj_acc.word + '.\n')
                elif inflected:
                    f.write(s + ' ' + iv + ' ' + o + '\t' + s.capitalize() + ' ' + inflected.word +\
                            ' ' + o + '.\n')
                elif obj_acc:
                    f.write(s + ' ' + iv + ' ' + o + '\t' + s.capitalize() + ' ' + iv +\
                            ' ' + obj_acc.word + '.\n')
                else:
                    f.write(s + ' ' + iv + ' ' + o + '\t' + s.capitalize() + ' ' + iv + ' ' + o + '.\n')

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [06:27<00:00, 193.18s/it]


In [287]:
# test
obj = morph.parse("картошка")[0]
obj.inflect({'accs'}).word

'картошку'